In [1]:
import sys
sys.path.append('..') # Adds higher directory to python modules path.

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from preprocessing.preprocessing_en import preprocess

In [2]:
# data
df = preprocess('../../resources/uci-news-aggregator.csv')
df = df[['title', 'class']]

Loading data
Data loaded
Actual preprocessing start
Actual preprocessing end


In [3]:
X = df['title']
Y = df['class']

# train/test split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [4]:
# vectorization
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)
print(vectorizer.get_feature_names()[:10])

['000', '00007458', '001', '0019', '002', '003', '0030', '003shr', '004', '005']


In [5]:
# classification
log_reg = LogisticRegression()
log_reg.fit(X_train, Y_train)

C:\Users\mikol\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\mikol\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [6]:
print(log_reg.score(X_train, Y_train))
print(log_reg.score(X_test, Y_test))

0.9538846227824878
0.9416339188485394
